In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import os

import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42

# 현재 device 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
DATA_PATH = '/content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/Naver_shopping_크롤링/Modeling/한수길/data/'

In [5]:
DATA_PATH_model = '/content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/Naver_shopping_크롤링/Modeling/한수길/data/감정분류가중치/'

In [6]:
model_name = "jaehyeong/koelectra-base-v3-generalized-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [7]:
test = pd.read_csv(f"{DATA_PATH}멀티레이블링완료.csv")
test

,content,score,date,brand,가격,세정력/거품,향기,트러블/진정,배송/서비스/고객대응,탈모 효과 및 쿨링감,머릿결*,가격추가,세정력추가,향기추가,트러블추가,배송추가,탈모효과추가,머릿결추가
0,만반잔당갈살살칸캉차,1,21.10.09.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1,펌프가 부러졌어요...이거 빼면 정말 좋은데..,1,21.08.16.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0
2,똑같네 똑같아 ㅡㅡ,1,20.10.19.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
3,좋다고 해서 샀는데... 좋은지도 나쁜지도 모르겠어요~ 아모레는 동물실험을 필요에 ...,1,20.06.14.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
4,평이 좋아서 샀는데 안맞네요,2,23.08.12.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84779,아들이 잘 쓰고있어요,5,21.09.18.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
84780,써볼께요ㅎㅎ써볼께요,5,21.09.18.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
84781,맛있고 배송도 빠르고 넘 넘 좋아요~,5,21.09.17.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0
84782,역시 닥터그루트닙니당,5,21.09.17.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0


In [8]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, token_type_ids, y_1=None, y_2=None):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.y_1 = y_1


    def __len__(self):
        return self.input_ids.shape[0]

    def __getitem__(self, idx):
        item = {}
        item['input_ids'] = torch.tensor(self.input_ids[idx])
        item['attention_mask'] = torch.tensor(self.attention_mask[idx])
        item['token_type_ids'] = torch.tensor(self.token_type_ids[idx])
        if self.y_1 is not None:
            item['y_1'] = torch.Tensor(self.y_1[idx])

        return item

In [9]:
class Net(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.output_layer = torch.nn.Linear( self.model.config.hidden_size , 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        x = self.model(input_ids, attention_mask, token_type_ids)
        x = self.output_layer(x[0][:,0])
        return x

In [10]:
def train_loop(dataloader, model, bce_loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 모델 객체를 학습모드로 전환
    for batch in tqdm(dataloader):
        pred = model(batch["input_ids"].to(device), batch['attention_mask'].to(device), batch['token_type_ids'].to(device))

        # 이진분류 loss
        bce_loss = bce_loss_fn(pred, batch["y_1"].to(device))

        optimizer.zero_grad()
        bce_loss.backward()
        optimizer.step()

        epoch_loss += bce_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [11]:
@torch.no_grad()
def test_loop(dataloader,model,bce_loss_fn, device):
    model.eval() # 평가 모드
    sig = torch.nn.Sigmoid()
    pred_1_list = []

    epoch_loss = 0
    for batch in tqdm(dataloader):
        pred = model(batch["input_ids"].to(device), batch['attention_mask'].to(device), batch['token_type_ids'].to(device))

        # 검증 평가할 경우
        if batch.get("y_1") is not None:
            bce_loss = bce_loss_fn(pred, batch["y_1"].to(device))

            epoch_loss += bce_loss.item()

        # 예측값 만들기
        pred_1 = sig(pred).to("cpu").numpy()

        pred_1_list.append(pred_1)

    pred_1 = np.concatenate(pred_1_list)
    epoch_loss /= len(dataloader)
    return epoch_loss, pred_1

In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score

batch_size = 8
loss_fn = torch.nn.BCEWithLogitsLoss()
epochs = 100
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)

In [13]:
test_1 = test[test['향기추가'] == 1]
test_1

,content,score,date,brand,가격,세정력/거품,향기,트러블/진정,배송/서비스/고객대응,탈모 효과 및 쿨링감,머릿결*,가격추가,세정력추가,향기추가,트러블추가,배송추가,탈모효과추가,머릿결추가
116,아직 얼마안되 탈모효과는 모르겠지만 순한거 같아요. 거품이 굉장히 부드러워요. 손...,3,20.05.22.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,0,0,0,0
239,이거 쓰고 그런지는 모르겠는데... 집 욕실 하수구가 너무 잘 막혀요 트린트먼트랑 ...,3,21.06.02.,이지마인드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0
266,개운해요 엘리자베스아덴 그린티향이랑 비슷해오,3,21.01.15.,이지마인드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0
270,잘쓰고 있습니다 좋네여 다만 향이 민감성 있으신분들은 조심,3,20.12.22.,이지마인드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0
273,냄새도 나쁘지 않고 용량도 좋고 다좋은데 깨져서 왔어요 흐르고.. 그래서 뚜껑도 안...,3,20.12.11.,이지마인드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84670,냄새가 상큼하고 가려움이 덜함,5,21.12.21.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,1,0,0,0
84734,향도 좋고 맘에드네요,5,21.10.27.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0
84736,힘이없는 모발용과 지성용은 색 향 다 달라요지성용은 진한갈색에 레몬향인것같아요비싸지...,5,21.10.25.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0
84747,아주좋아요 향도굿굿,5,21.10.12.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0


In [14]:
test_token = tokenizer(test_1['content'].tolist(), padding="max_length", max_length=512, truncation=True)

test_input_ids = np.array(test_token['input_ids'])
test_attention_mask = np.array(test_token['attention_mask'])
test_token_type_ids = np.array(test_token['token_type_ids'])

test_input_ids.shape, test_attention_mask.shape, test_token_type_ids.shape

((11500, 512), (11500, 512), (11500, 512))

In [15]:
batch_size = 64

In [16]:
test_dt = ReviewDataset(test_input_ids, test_attention_mask, test_token_type_ids)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=batch_size, shuffle = False)

pred_list = []
for i in range(n_splits):
    model = Net(model_name).to(device)
    state_dict = torch.load(f"{DATA_PATH_model}model_emotion_scent_{i}.pth") # 가중치 불러오기
    model.load_state_dict(state_dict) # 모델에 가중치 세팅

    _, pred = test_loop(test_dl, model, loss_fn, device)
    pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

In [17]:
a = (pred > 0.5).astype(int)

In [18]:
a = np.where(a == 0, 1, 0)

In [19]:
test.loc[test['향기추가'] == 1, '향기'] = a
test

,content,score,date,brand,가격,세정력/거품,향기,트러블/진정,배송/서비스/고객대응,탈모 효과 및 쿨링감,머릿결*,가격추가,세정력추가,향기추가,트러블추가,배송추가,탈모효과추가,머릿결추가
0,만반잔당갈살살칸캉차,1,21.10.09.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1,펌프가 부러졌어요...이거 빼면 정말 좋은데..,1,21.08.16.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0
2,똑같네 똑같아 ㅡㅡ,1,20.10.19.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
3,좋다고 해서 샀는데... 좋은지도 나쁜지도 모르겠어요~ 아모레는 동물실험을 필요에 ...,1,20.06.14.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
4,평이 좋아서 샀는데 안맞네요,2,23.08.12.,라보에이치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84779,아들이 잘 쓰고있어요,5,21.09.18.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
84780,써볼께요ㅎㅎ써볼께요,5,21.09.18.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
84781,맛있고 배송도 빠르고 넘 넘 좋아요~,5,21.09.17.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0
84782,역시 닥터그루트닙니당,5,21.09.17.,닥터그루트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0


In [20]:
test[(test['향기'] == 1) | (test['향기'] == 0)]

,content,score,date,brand,가격,세정력/거품,향기,트러블/진정,배송/서비스/고객대응,탈모 효과 및 쿨링감,머릿결*,가격추가,세정력추가,향기추가,트러블추가,배송추가,탈모효과추가,머릿결추가
116,아직 얼마안되 탈모효과는 모르겠지만 순한거 같아요. 거품이 굉장히 부드러워요. 손...,3,20.05.22.,라보에이치,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,1,1,0,0,0,0
239,이거 쓰고 그런지는 모르겠는데... 집 욕실 하수구가 너무 잘 막혀요 트린트먼트랑 ...,3,21.06.02.,이지마인드,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0
266,개운해요 엘리자베스아덴 그린티향이랑 비슷해오,3,21.01.15.,이지마인드,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0
270,잘쓰고 있습니다 좋네여 다만 향이 민감성 있으신분들은 조심,3,20.12.22.,이지마인드,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0
273,냄새도 나쁘지 않고 용량도 좋고 다좋은데 깨져서 왔어요 흐르고.. 그래서 뚜껑도 안...,3,20.12.11.,이지마인드,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84670,냄새가 상큼하고 가려움이 덜함,5,21.12.21.,닥터그루트,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,1,0,0,0
84734,향도 좋고 맘에드네요,5,21.10.27.,닥터그루트,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0
84736,힘이없는 모발용과 지성용은 색 향 다 달라요지성용은 진한갈색에 레몬향인것같아요비싸지...,5,21.10.25.,닥터그루트,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0
84747,아주좋아요 향도굿굿,5,21.10.12.,닥터그루트,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0


In [21]:
test.to_csv(f'{DATA_PATH}향기최종labeling_완료.csv', index=False, encoding='utf-8-sig')